# II. Merge 8b-chat-doctor avec Llama3 

In [2]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [1]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF ")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
# Importation des bibliothèques nécessaires
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format

2024-06-14 12:51:42.338554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 12:51:42.338663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 12:51:42.469086: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ModuleNotFoundError: No module named 'peft'

In [5]:
# Définir les chemins des modèles
base_model_path = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
new_model_path = "PierreMaxime/llama-3-8b-chat-doctor"

tokenizer = AutoTokenizer.from_pretrained(base_model_path)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    return_dict=True,
    low_cpu_mem_usage=True, # Pour l'optimisation de la mémoire CPU
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

base_model, tokenizer = setup_chat_format(base_model, tokenizer)

# Charger l'adaptateur et le fusionner avec le modèle de base
peft_model = PeftModel.from_pretrained(base_model, new_model_path)
merged_model = peft_model.merge_and_unload()

# Sauvegarder le modèle fusionné pour une utilisation ultérieure
output_model_path = "/kaggle/working/llama-3-8b-chat-doctor-merged"
merged_model.save_pretrained(output_model_path, safe_serialization=False)
tokenizer.save_pretrained(output_model_path)


print("Le modèle fusionné a été sauvegardé à l'emplacement :", output_model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Le modèle fusionné a été sauvegardé à l'emplacement : /kaggle/working/llama-3-8b-chat-doctor-merged


In [8]:
# Pousser le modèle sur Hugging Face 
from huggingface_hub import login, upload_folder, create_repo

repo_id = "PierreMaxime/llama-3-8b-chat-doctor-merged"
create_repo(repo_id, exist_ok=True, private=False) 

RepoUrl('https://huggingface.co/PierreMaxime/llama-3-8b-chat-doctor-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='PierreMaxime/llama-3-8b-chat-doctor-merged')

In [9]:
from huggingface_hub import login, upload_folder

upload_folder(
    repo_id=repo_id,
    folder_path=output_model_path,
    commit_message="Ajout du modèle fusionné",
    use_auth_token=True
)

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/PierreMaxime/llama-3-8b-chat-doctor-merged/commit/99346f862c60c91461034e73251500ed43182e5a', commit_message='Ajout du modèle fusionné', commit_description='', oid='99346f862c60c91461034e73251500ed43182e5a', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
output_model_path

'/kaggle/working/llama-3-8b-chat-doctor-merged'